Check Create_Own_pkl_file.ipynb file to get the pkl file used here.

In [9]:
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
cv=TfidfVectorizer(max_features=5000,stop_words='english')


def reduce_memory(df):
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col] = df[col].astype('float32')
        if df[col].dtype == 'int64':
            df[col] = df[col].astype('int32')
    return df

games = reduce_memory(pd.read_csv('./final_dataset.csv'))
n_recommendation = 20
vectors = pickle.load(open("./vectors_final_forDesc_tfid.pkl",'rb'))

# similarity=pickle.load(open("./similarity_forDesc.pkl",'rb'))

# def recommend(game):
#     index = games[games['title'] == game].index[0]
#     sim_scores = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
#     print(sim_scores)
#     game_lists=[]
#     for i in sim_scores[1:n_recommendation]:
#         game_lists.append(games.iloc[i[0]].title)
#     return game_lists
def recommend(game):
    index = games[games['title'] == game].index[0]
    

    item_vector = vectors[index]
    similarities = cosine_similarity(item_vector, vectors).flatten()
    recommended_indices = similarities.argsort()[::-1]  # Get the indices of the most similar items
    game_lists=[]
    for i in recommended_indices[1:n_recommendation]:
        game_lists.append(games.iloc[i].title)
    return (game_lists)

# recommend('God Sword')

## Imroved Desc now with own Desc

In [15]:
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
import re

def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)  # Removes all non-alphanumeric characters except spaces
    
    # Apply stemming
    text = ' '.join([stemmer.stem(word) for word in text.split()])
    
    return text


def reduce_memory(df):
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col] = df[col].astype('float32')
        if df[col].dtype == 'int64':
            df[col] = df[col].astype('int32')
    return df

# Load data and precomputed vectors
tfidfV = pickle.load(open("tfidf_vectorizer.pkl", 'rb'))#yo fit on dataset which will be used to transfrom the input desc. yo chai naya pkl file 
games = reduce_memory(pd.read_csv('./final_dataset.csv'))
vectors = pickle.load(open("./vectors_final_forDesc_tfid.pkl", 'rb'))#yo fit_transfrom on dataset 
n_recommendation = 20


def recommend(input_text):
    try:
        # Preprocess the input text
        processed_text = preprocess_text(input_text)
        
        # Check if the processed input matches a game title
        if processed_text in games['title'].str.lower().values:
            index = games[games['title'].str.lower() == processed_text].index[0]
            item_vector = vectors[index]
        else:
            # Treat processed input as a description and transform it to a vector
            input_vector = tfidfV.transform([processed_text])
            item_vector = input_vector.toarray()  # Convert sparse matrix to dense format for compatibility
        
        # Calculate cosine similarities
        similarities = cosine_similarity(item_vector, vectors).flatten()
        recommended_indices = similarities.argsort()[::-1]  # Sort by similarity score in descending order

        # Retrieve recommended games
        game_lists = []
        for i in recommended_indices[1:n_recommendation]:
            game_lists.append(games.iloc[i].title)
        return game_lists

    except Exception as e:
        return f"An error occurred: {e}"


In [23]:
recommend("rpg mmorpg")

['MMORPG Tycoon 2',
 '已下架',
 '原始传奇',
 '云巅',
 'Inferna',
 '龙魂大冒险',
 '大话江湖',
 'Pandum online',
 'Project: Gorgon',
 'Legend of Edda Brasil',
 'Zezenia Online',
 'RPG MO',
 '御龙在天-平衡国战版',
 'Eternal Quest - 2D MMORPG',
 'ASU Global',
 'NosTale - Anime MMORPG',
 'Kaion Tale MMORPG',
 'Hero Plus',
 'Entasy Online']

In [20]:
recommend("Call of Duty®: Black Ops Cold War")

['Space Ops VR: Reloaded',
 'Wings Over Europe',
 'Call of Duty® 2',
 'US and THEM',
 'Command Ops 2 Core Game',
 'Call of Duty: United Offensive',
 'Men of War: Assault Squad 2 - Cold War',
 'The Bureau: XCOM Declassified',
 'Theatre of War 3: Korea',
 'Darkest Hour: A Hearts of Iron Game',
 'Supreme Ruler: Cold War',
 'SWAT 3: Tactical Game of the Year Edition',
 'The Cursed Tower',
 'Supreme Ruler Ultimate',
 "Alekhine's Gun",
 'The Cold War Era',
 'Battlefield: Bad Company 2 Vietnam',
 'Men of War: Vietnam',
 'Follow The Crown']

In [16]:
recommend("Half-Life 2: Episode One")

['Half-Life 2: Episode Two',
 "Half-Life 2: Genry's Great Escape From City 13",
 'Half-Life: Before',
 'Half-Life 2',
 'Half-Life: Source',
 'Half-Life 2: Update',
 'Half-Life: Restored',
 'Requiem Of Science',
 'Half-Life Deathmatch: Source',
 'Half-Life 2: VR Mod - Episode One',
 'Entropy : Zero',
 'Half-Life: Blue Shift',
 'Half-Life: Alyx - Final Hours',
 'Half-Life 2: Lost Coast',
 'Halfquake Trilogy',
 'Synergy',
 'Half-Life: MMod',
 'Half-Life: VR Mod',
 'Transmissions: Element 120']

In [12]:
def get_game_recommendations(game_name):
    n=10
    game_similarity_df = pickle.load(open("./game_similarity_df.pkl",'rb'))
    mapped_df = pickle.load(open("./mapped_df.pkl",'rb'))
    # Check if the game name exists in the filtered_df_player_count
    if game_name not in mapped_df['title'].values:
        print("The game lacks rating. Unable to find recommendations.")
        return

    
    # Get the app_id of the input game
    app_id = mapped_df.loc[mapped_df['title'] == game_name, 'app_id'].values[0]
    
    # Check if the app_id is in the similarity matrix
    if app_id not in game_similarity_df.index:
        print("No similar games found/ Low rated game ")
        return
    
    # Get similarity scores for the game and sort them in descending order
    similarity_scores = game_similarity_df.loc[app_id].sort_values(ascending=False)
    
    # Get top n similar app_ids (excluding the game itself)
    top_app_ids = similarity_scores.iloc[1:n+1].index  # Exclude the first as it's the game itself
    
    # Map app_ids to game titles
    recommendations = mapped_df[mapped_df['app_id'].isin(top_app_ids)]['title'].tolist()
    
    return (recommendations)

In [39]:
ground_truth = {
    "Call of Duty® 2": ["Call of Duty®", "Call of Duty®: Modern Warfare® Remastered (2017)", 
                        "Battle For The Sun", "Call of Duty: United Offensive","Verdun",'Call of Duty® 4: Modern Warfare® (2007)'],
    "Half-Life 2: Episode One": ["Half-Life 2","Half-Life 2: Episode Two", "Half-Life: Source","Half-Life: Before","Half-Life 2: Genry's Great Escape From City 13","Half-Life 2: VR Mod - Episode One"],  # Relevant items for Half-Life
    "Battlefield: Bad Company™ 2": ["Battlefield 4™","Battlefield™ 2042","Combat Cycle","Heroic Armored Company","Vanguard: Normandy 1944","Battlefield: Bad Company 2 Vietnam"],
    "Batman: Arkham City - Game of the Year Edition": ["Batman: Arkham Asylum Game of the Year Edition","Batman™: Arkham Origins","Injustice: Gods Among Us Ultimate Edition","Batman™: Arkham Origins Blackgate - Deluxe Edition","PowersVR"]
}

In [42]:
recommend("Call of Duty® 2")

['Call of Duty: United Offensive',
 'Call of Duty®',
 'Battle For The Sun',
 'Call of Duty® 4: Modern Warfare® (2007)',
 'SWAT 3: Tactical Game of the Year Edition',
 'Call of Duty®: Modern Warfare® Remastered (2017)',
 'Warrior Paint - 2005 GOTY Edition',
 'Land of War - The Beginning',
 'Day of Defeat: Source',
 'Call of Duty®: Ghosts',
 'Verdun',
 'Fog Of War - Free Edition',
 'World War II GI',
 'Red Orchestra: Ostfront 41-45',
 'Blitzkrieg 2 Anthology',
 'Toy Soldiers',
 'Wheels of Duty',
 'Honor and Duty: D-Day',
 'Day of Infamy']

In [27]:
def calculate_precision_recall(input_title, recommended_titles, ground_truth, top_n=3):
    """
    Calculate precision, recall, and other metrics for a single input title.
    """
    # Get the relevant items from the ground truth for this test item
    relevant_items = ground_truth.get(input_title, [])
    if not relevant_items:
        return {"precision": 0, "recall": 0, "true_positives": 0, "false_positives": 0, "false_negatives": 0}
    # Consider only the top N recommended items
    top_recommended = recommended_titles[:top_n]
    # Calculate true positives, false positives, and false negatives
    true_positives = len(set(relevant_items) & set(top_recommended))
    false_positives = len(set(top_recommended) - set(relevant_items))
    false_negatives = len(set(relevant_items) - set(top_recommended))
    # Calculate precision and recall
    precision = true_positives / len(top_recommended) if len(top_recommended) > 0 else 0
    recall = true_positives / len(relevant_items) if len(relevant_items) > 0 else 0
    return {
        "precision": precision,
        "recall": recall,
        "true_positives": true_positives,
        "false_positives": false_positives,
        "false_negatives": false_negatives,
    }


In [56]:
input_title = "Half-Life"
# recommended_titles = recommend(input_title)
recommended_titles = get_game_recommendations("Half-Life")

print(type(recommended_titles));

# Calculate precision and recall for the recommendation
metrics = calculate_precision_recall(input_title, recommended_titles, ground_truth, top_n=8)

print(f"Recommendations for '{input_title}':")
print(recommended_titles)
print(f"Precision: {metrics['precision']}")
print(f"Recall: {metrics['recall']}")
precision = metrics['precision']
recall = metrics['recall']

# Calculate F1 score
if precision + recall > 0:
    f1_score = 2 * (precision * recall) / (precision + recall)
else:
    f1_score = 0.0

print("Fi Score:", f1_score)



<class 'list'>
['Half-Life 2', 'Half-Life 2: Episode One', 'Half-Life 2: Episode Two', 'Portal', 'Team Fortress 2', 'Left 4 Dead', 'Portal 2']
['Half-Life 2', 'Counter-Strike: Source', 'Half-Life 2: Episode One', 'Portal', 'Half-Life 2: Episode Two', 'Team Fortress 2', 'Left 4 Dead', 'Portal 2']
Recommendations for 'Half-Life':
['Half-Life 2', 'Counter-Strike: Source', 'Half-Life 2: Episode One', 'Portal', 'Half-Life 2: Episode Two', 'Team Fortress 2', 'Left 4 Dead', 'Portal 2', 'POSTAL 2', 'Black Mesa']
Precision: 0.875
Recall: 1.0
0.9333333333333333


In [41]:
all_titles = ["Call of Duty® 2", "Half-Life 2: Episode One","Battlefield: Bad Company™ 2","Batman: Arkham City - Game of the Year Edition"]
f1_scores = []
total_tp, total_fp, total_fn = 0, 0, 0  # For micro-average
for input_title in all_titles:
    # Get recommendations
    recommended_titles = recommend(input_title)
    # Calculate precision and recall
    metrics = calculate_precision_recall(input_title, recommended_titles, ground_truth, top_n=6)
    precision = metrics['precision']
    recall = metrics['recall']
    # Calculate F1 score for this title
    if precision + recall > 0:
        f1 = 2 * (precision * recall) / (precision + recall)
    else:
        f1 = 0.0
    # Add F1 score to list for macro-average
    f1_scores.append(f1)
    # Aggregate TP, FP, FN for micro-average
    total_tp += metrics['true_positives']
    total_fp += metrics['false_positives']
    total_fn += metrics['false_negatives']
# Micro-Averaged F1
if total_tp + total_fp > 0 and total_tp + total_fn > 0:
    micro_precision = total_tp / (total_tp + total_fp)
    micro_recall = total_tp / (total_tp + total_fn)
    micro_f1 = 2 * (micro_precision * micro_recall) / (micro_precision + micro_recall)
else:
    micro_f1 = 0.0
# print(f"Macro-Averaged F1: {macro_f1}")
print(f"Precision: {micro_precision}" )
print(f"Recall: {micro_recall}" )
print(f"Averaged F1 Score: {micro_f1}")


Precision: 0.8333333333333334
Recall: 0.8695652173913043
Averaged F1 Score: 0.851063829787234


['Half-Life 2',
 'Counter-Strike: Source',
 'Half-Life 2: Episode One',
 'Portal',
 'Half-Life 2: Episode Two',
 'Team Fortress 2',
 'Left 4 Dead',
 'Portal 2',
 'POSTAL 2',
 'Black Mesa']